In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#X_train = X_train - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]
#X_test = X_test - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]

X_train_full, y_train_full = X_train, y_train

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]



c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ferbi\appd

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_inputs = X_train.shape[1]
n_hidden1 = 1024
n_hidden2 = 1024
n_hidden3 = 1024
n_hidden4 = 1024
n_hidden5 = 1024
n_outputs = 10

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4", activation=tf.nn.relu)
    hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5", activation=tf.nn.relu)
    logits = neuron_layer(hidden5, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
hidden3_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/kernel:0")
hidden4_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden4/kernel:0")
hidden5_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden5/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")
    
def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 90*tf.pow(W, 2)))))
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 8
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden3_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden4_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden5_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(hidden3_W), \
                                  tf.count_nonzero(hidden4_W), \
                                  tf.count_nonzero(hidden5_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden3_W) < epsVal, hidden3_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden4_W) < epsVal, hidden4_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden5_W) < epsVal, hidden5_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

reg_summary = tf.summary.scalar('Count_Reg', total_regvalue)
loss_summary = tf.summary.scalar('Loss_Total', loss_total)
err_summary = tf.summary.scalar('Class_Err_Perc', (1-accuracy)*100)
#file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, reg_loss_val, lambda1, acc_val = sess.run([loss_total, total_regvalue, lambdaParam, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        base_loss_val = loss_val - lambda1*reg_loss_val
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation losses: {:.4f}, {:.4f}, {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, base_loss_val, lambda1*reg_loss_val, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20190129210937.ckpt") # or better, use save_path
    n_batches = len(X_test) // 2000
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [3]:
for j in [-4.0, -4.2, -4.4, -4.6, -4.8, -5.0, -5.2, -5.4, -5.6, -5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
    for i in [1, 2, 3]:
        print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
        execute_code(rand_seed=i, lambda_exp=j)

RANDOM SEED:   1 	LAMBDA_EXP:   -4.0
Now: 20240508223216
0	Validation losses: 0.4898, 39.0635, 39.5533	Best Loss: 39.5533 (0)	Accuracy: 81.58%
1	Validation losses: 0.4641, 8.0973, 8.5614	Best Loss: 8.5614 (0)	Accuracy: 84.38%
2	Validation losses: 0.5082, 0.9317, 1.4400	Best Loss: 1.4400 (0)	Accuracy: 82.14%
3	Validation losses: 0.5412, 0.4196, 0.9608	Best Loss: 0.9608 (0)	Accuracy: 80.98%
4	Validation losses: 0.5126, 0.3580, 0.8706	Best Loss: 0.8706 (0)	Accuracy: 82.06%
5	Validation losses: 0.5506, 0.2821, 0.8327	Best Loss: 0.8327 (0)	Accuracy: 79.82%
6	Validation losses: 0.4583, 0.2126, 0.6710	Best Loss: 0.6710 (0)	Accuracy: 84.14%
7	Validation losses: 0.5783, 0.2073, 0.7856	Best Loss: 0.6710 (1)	Accuracy: 79.92%
8	Validation losses: 0.5476, 0.1728, 0.7203	Best Loss: 0.6710 (2)	Accuracy: 81.10%
9	Validation losses: 0.6699, 0.2107, 0.8806	Best Loss: 0.6710 (3)	Accuracy: 77.40%
10	Validation losses: 0.4892, 0.1637, 0.6530	Best Loss: 0.6530 (0)	Accuracy: 83.08%
11	Validation losses: 0.55

23	Validation losses: 2.3061, 0.4089, 2.7150	Best Loss: 0.6355 (11)	Accuracy: 9.86%
24	Validation losses: 2.3045, 0.3938, 2.6984	Best Loss: 0.6355 (12)	Accuracy: 9.86%
25	Validation losses: 2.3053, 0.3889, 2.6942	Best Loss: 0.6355 (13)	Accuracy: 9.86%
26	Validation losses: 2.3057, 0.3859, 2.6916	Best Loss: 0.6355 (14)	Accuracy: 9.14%
27	Validation losses: 2.3027, 0.3836, 2.6863	Best Loss: 0.6355 (15)	Accuracy: 9.76%
28	Validation losses: 2.3044, 0.3820, 2.6864	Best Loss: 0.6355 (16)	Accuracy: 9.80%
29	Validation losses: 2.3041, 0.3805, 2.6846	Best Loss: 0.6355 (17)	Accuracy: 10.08%
30	Validation losses: 2.3037, 0.3794, 2.6831	Best Loss: 0.6355 (18)	Accuracy: 10.08%
31	Validation losses: 2.3028, 0.3785, 2.6812	Best Loss: 0.6355 (19)	Accuracy: 11.12%
32	Validation losses: 2.3039, 0.3776, 2.6815	Best Loss: 0.6355 (20)	Accuracy: 9.14%
Early stopping!
Total running time:	 2648
INFO:tensorflow:Restoring parameters from ./models/model_20240509000218.ckpt
Batches remaining:     5
Batches remai

13	Validation losses: 0.4760, 0.1195, 0.5955	Best Loss: 0.5955 (0)	Accuracy: 82.88%
14	Validation losses: 0.5041, 0.1436, 0.6477	Best Loss: 0.5955 (1)	Accuracy: 83.72%
15	Validation losses: 0.4877, 0.1357, 0.6234	Best Loss: 0.5955 (2)	Accuracy: 83.72%
16	Validation losses: 0.4470, 0.1340, 0.5809	Best Loss: 0.5809 (0)	Accuracy: 84.26%
17	Validation losses: 0.4524, 0.1355, 0.5879	Best Loss: 0.5809 (1)	Accuracy: 84.34%
18	Validation losses: 0.4458, 0.1360, 0.5818	Best Loss: 0.5809 (2)	Accuracy: 83.82%
19	Validation losses: 0.5257, 0.1379, 0.6636	Best Loss: 0.5809 (3)	Accuracy: 82.72%
20	Validation losses: 0.6440, 0.1595, 0.8035	Best Loss: 0.5809 (4)	Accuracy: 79.64%
21	Validation losses: 0.5080, 0.1551, 0.6631	Best Loss: 0.5809 (5)	Accuracy: 83.00%
22	Validation losses: 0.5806, 0.1731, 0.7537	Best Loss: 0.5809 (6)	Accuracy: 81.66%
23	Validation losses: 0.4649, 0.1574, 0.6223	Best Loss: 0.5809 (7)	Accuracy: 83.86%
24	Validation losses: 0.5085, 0.1584, 0.6669	Best Loss: 0.5809 (8)	Accuracy:

46	Validation losses: 0.3787, 0.1140, 0.4927	Best Loss: 0.4906 (6)	Accuracy: 86.66%
47	Validation losses: 0.3870, 0.1139, 0.5008	Best Loss: 0.4906 (7)	Accuracy: 86.08%
48	Validation losses: 0.3790, 0.1135, 0.4926	Best Loss: 0.4906 (8)	Accuracy: 86.82%
49	Validation losses: 0.3922, 0.1143, 0.5065	Best Loss: 0.4906 (9)	Accuracy: 86.10%
50	Validation losses: 0.3865, 0.1146, 0.5011	Best Loss: 0.4906 (10)	Accuracy: 86.40%
51	Validation losses: 0.3927, 0.1144, 0.5071	Best Loss: 0.4906 (11)	Accuracy: 86.52%
52	Validation losses: 0.3959, 0.1140, 0.5099	Best Loss: 0.4906 (12)	Accuracy: 85.86%
53	Validation losses: 0.4055, 0.1150, 0.5205	Best Loss: 0.4906 (13)	Accuracy: 85.72%
54	Validation losses: 0.4021, 0.1150, 0.5170	Best Loss: 0.4906 (14)	Accuracy: 85.92%
55	Validation losses: 0.3934, 0.1164, 0.5098	Best Loss: 0.4906 (15)	Accuracy: 86.28%
56	Validation losses: 0.3986, 0.1166, 0.5151	Best Loss: 0.4906 (16)	Accuracy: 85.78%
57	Validation losses: 0.4280, 0.1172, 0.5452	Best Loss: 0.4906 (17)	A

37	Validation losses: 0.3737, 0.0873, 0.4610	Best Loss: 0.4610 (0)	Accuracy: 86.90%
38	Validation losses: 0.3758, 0.0866, 0.4624	Best Loss: 0.4610 (1)	Accuracy: 86.58%
39	Validation losses: 0.3648, 0.0863, 0.4511	Best Loss: 0.4511 (0)	Accuracy: 86.78%
40	Validation losses: 0.3637, 0.0858, 0.4495	Best Loss: 0.4495 (0)	Accuracy: 86.94%
41	Validation losses: 0.3634, 0.0858, 0.4493	Best Loss: 0.4493 (0)	Accuracy: 87.10%
42	Validation losses: 0.3677, 0.0861, 0.4538	Best Loss: 0.4493 (1)	Accuracy: 86.88%
43	Validation losses: 0.3674, 0.0862, 0.4535	Best Loss: 0.4493 (2)	Accuracy: 86.68%
44	Validation losses: 0.3790, 0.0864, 0.4654	Best Loss: 0.4493 (3)	Accuracy: 86.40%
45	Validation losses: 0.3782, 0.0864, 0.4646	Best Loss: 0.4493 (4)	Accuracy: 86.70%
46	Validation losses: 0.3760, 0.0867, 0.4626	Best Loss: 0.4493 (5)	Accuracy: 87.04%
47	Validation losses: 0.3789, 0.0872, 0.4661	Best Loss: 0.4493 (6)	Accuracy: 86.82%
48	Validation losses: 0.3895, 0.0865, 0.4760	Best Loss: 0.4493 (7)	Accuracy:

...

12	Validation losses: 0.3429, 0.2520, 0.5949	Best Loss: 0.5663 (4)	Accuracy: 88.56%
13	Validation losses: 0.3798, 0.2476, 0.6273	Best Loss: 0.5663 (5)	Accuracy: 88.02%
14	Validation losses: 0.3800, 0.2433, 0.6233	Best Loss: 0.5663 (6)	Accuracy: 86.18%
15	Validation losses: 0.3122, 0.2390, 0.5512	Best Loss: 0.5512 (0)	Accuracy: 89.20%
16	Validation losses: 0.3322, 0.2347, 0.5669	Best Loss: 0.5512 (1)	Accuracy: 89.12%
17	Validation losses: 0.3079, 0.2304, 0.5382	Best Loss: 0.5382 (0)	Accuracy: 89.62%
18	Validation losses: 0.3602, 0.2260, 0.5862	Best Loss: 0.5382 (1)	Accuracy: 88.78%
19	Validation losses: 0.3261, 0.2222, 0.5483	Best Loss: 0.5382 (2)	Accuracy: 88.94%
20	Validation losses: 0.3334, 0.2183, 0.5517	Best Loss: 0.5382 (3)	Accuracy: 88.22%
21	Validation losses: 0.3610, 0.2137, 0.5747	Best Loss: 0.5382 (4)	Accuracy: 88.90%
22	Validation losses: 0.3268, 0.2090, 0.5358	Best Loss: 0.5358 (0)	Accuracy: 89.40%
23	Validation losses: 0.3261, 0.2043, 0.5305	Best Loss: 0.5305 (0)	Accuracy:

4	Validation losses: 0.3256, 0.1780, 0.5035	Best Loss: 0.5035 (0)	Accuracy: 87.68%
5	Validation losses: 0.4323, 0.1775, 0.6098	Best Loss: 0.5035 (1)	Accuracy: 84.96%
6	Validation losses: 0.2943, 0.1770, 0.4712	Best Loss: 0.4712 (0)	Accuracy: 88.90%
7	Validation losses: 0.4261, 0.1764, 0.6025	Best Loss: 0.4712 (1)	Accuracy: 85.36%
8	Validation losses: 0.2978, 0.1757, 0.4735	Best Loss: 0.4712 (2)	Accuracy: 89.12%
9	Validation losses: 0.3288, 0.1750, 0.5038	Best Loss: 0.4712 (3)	Accuracy: 88.78%
10	Validation losses: 0.3177, 0.1741, 0.4918	Best Loss: 0.4712 (4)	Accuracy: 88.98%
11	Validation losses: 0.3010, 0.1733, 0.4743	Best Loss: 0.4712 (5)	Accuracy: 89.38%
12	Validation losses: 0.3832, 0.1725, 0.5557	Best Loss: 0.4712 (6)	Accuracy: 86.56%
13	Validation losses: 0.4189, 0.1716, 0.5905	Best Loss: 0.4712 (7)	Accuracy: 86.00%
14	Validation losses: 0.3152, 0.1707, 0.4859	Best Loss: 0.4712 (8)	Accuracy: 89.12%
15	Validation losses: 0.3646, 0.1699, 0.5345	Best Loss: 0.4712 (9)	Accuracy: 88.38

Early stopping!
Total running time:	 2407
INFO:tensorflow:Restoring parameters from ./models/model_20240510213407.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    11.7400002480%
RANDOM SEED:   1 	LAMBDA_EXP:   -6.8
Now: 20240510221415
0	Validation losses: 0.4043, 0.1131, 0.5174	Best Loss: 0.5174 (0)	Accuracy: 84.98%
1	Validation losses: 0.3515, 0.1131, 0.4646	Best Loss: 0.4646 (0)	Accuracy: 87.50%
2	Validation losses: 0.3775, 0.1131, 0.4907	Best Loss: 0.4646 (1)	Accuracy: 86.38%
3	Validation losses: 0.3490, 0.1132, 0.4622	Best Loss: 0.4622 (0)	Accuracy: 87.00%
4	Validation losses: 0.3217, 0.1133, 0.4350	Best Loss: 0.4350 (0)	Accuracy: 88.08%
5	Validation losses: 0.4084, 0.1133, 0.5218	Best Loss: 0.4350 (1)	Accuracy: 85.92%
6	Validation losses: 0.2920, 0.1134, 0.4054	Best Loss: 0.4054 (0)	Accuracy: 89.16%
7	Validation losses: 0.3567, 0.1136, 0.4703	Best Loss: 0.4054 (1)	Accuracy: 86.58%


Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    11.8200004101%
RANDOM SEED:   1 	LAMBDA_EXP:   -7.0
Now: 20240511001215
0	Validation losses: 0.4056, 0.0714, 0.4769	Best Loss: 0.4769 (0)	Accuracy: 85.00%
1	Validation losses: 0.3492, 0.0714, 0.4206	Best Loss: 0.4206 (0)	Accuracy: 87.70%
2	Validation losses: 0.3814, 0.0715, 0.4530	Best Loss: 0.4206 (1)	Accuracy: 85.98%
3	Validation losses: 0.3429, 0.0717, 0.4146	Best Loss: 0.4146 (0)	Accuracy: 87.48%
4	Validation losses: 0.3175, 0.0718, 0.3894	Best Loss: 0.3894 (0)	Accuracy: 88.26%
5	Validation losses: 0.4287, 0.0721, 0.5007	Best Loss: 0.3894 (1)	Accuracy: 85.42%
6	Validation losses: 0.2945, 0.0723, 0.3668	Best Loss: 0.3668 (0)	Accuracy: 88.92%
7	Validation losses: 0.3416, 0.0726, 0.4142	Best Loss: 0.3668 (1)	Accuracy: 87.60%
8	Validation losses: 0.3033, 0.0730, 0.3762	Best Loss: 0.3668 (2)	Accuracy: 88.96%
9	Validation losses: 0.3535, 0.0733,

Batches remaining:     2
Batches remaining:     1
Test error rate:    11.8099999428%
